<a href="https://colab.research.google.com/github/AndreRab/Polish-poems-generation/blob/main/polish_poems_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs

In [ ]:
!pip install PyMuPDF
!pip install pdfplumber
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Succe

## Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import fitz
from io import BytesIO
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Data collecting

In [ ]:
RYMY_COLLECTIONS = [
    "https://wolnelektury.pl/api/collections/wiersze-dla-dzieci/",
    "https://wolnelektury.pl/api/collections/lirycy-mlodopolscy/",
    # "https://wolnelektury.pl/api/collections/piesni-patriotyczne/"
]

In [ ]:
def get_poem_links(url, limit=10000):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Błąd pobierania linków z: {url}")
        return []
    try:
        data = response.json()
    except Exception as e:
        print("Nie udało się zdekodować JSON:", e)
        return []
    books = data.get('books', [])[:limit]
    return [book['href'] for book in books]

In [ ]:
def get_pdf(url):
  response = requests.get(url)
  data = response.json()
  return data.get('pdf', None)

def clear_text(text):
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    content_lines = lines[6:-28]
    return "\n".join(content_lines)

def fix_corrupted_polish(text):
  replacements = {
      'గ': 'j',
      'ǳ': 'dz',
      'ǲ': 'Dz',
      'Ǳ': 'DZ',
      '': 'H',
      '': 'W',
      '': 'A',
      '': 'N',
      '': 'R',
      '': 'I',
      '': 'U',
      '': 'T',
      '': 'L',
      '': 'O',
      '': 'Z',
      '': 'M',
      '': 'C',
      '': 'E',
      '': 'D',
      '': 'Y',
      '': 'K',
      '': 'S',
      '': 'B',
      '': 'G',
      '': 'J',
      '': 'X',
      '': 'Q',
      '': 'V',
      '': 'HENRY',
      '': 'WALU',
      'ﬂ': 'fl',
      'ﬁ': 'fi',
      'ట': 'ś'
  }

  for wrong, correct in replacements.items():
      text = text.replace(wrong, correct)

  return text

def extract_data(section_url, limit=1000):
  data = []
  links = get_poem_links(section_url, limit)
  for link in tqdm(links):
    response = requests.get(get_pdf(link))
    pdf_data = BytesIO(response.content)
    try:
      doc = fitz.open(stream=pdf_data, filetype="pdf")
    except Exception:
      continue
    text = ""

    for page in doc:
        text += page.get_text()
    data.append(fix_corrupted_polish(clear_text(text)))
  return data

In [ ]:
data = []
for collection in RYMY_COLLECTIONS:
  data.extend(extract_data(collection))

100%|██████████| 131/131 [00:40<00:00,  3.21it/s]


In [ ]:
len(data)

150

In [ ]:
df = pd.DataFrame(
    data= {
        'text': data
    }
)

In [ ]:
def delete_first_line(text):
    index = text.find('\n')
    return text[index + len('\n'):]

df.text = df.map(lambda x: delete_first_line(x))

In [ ]:
from collections import Counter

def delete_duplicates_and_split(text):
    lines = text.splitlines()
    line_counts = Counter(lines)

    splited_parts = []
    current_part = []

    for line in lines:
        if line_counts[line] < 5:
            current_part.append(line)
        else:
            if current_part:  # Avoid empty entries
                splited_parts.append("\n".join(current_part))
                current_part = []

    if current_part:
        splited_parts.append("\n".join(current_part))

    return splited_parts

splited_data = []
for i, text in enumerate(df.text):
    splited_data += (delete_duplicates_and_split(text))

df = pd.DataFrame({'text': splited_data})
df

,text
0,"Pozwólcie, że wam przedstawię\nżółwia,\nco ma ..."
1,\nKogut\n— Kuku! Ryku! Kukuryku‼! —\njuż od ś...
2,\nJuż o świcie gna na łąkę —\nśmiało wita się...
3,"\nDba o zdrowie należycie,\nco dzień drepcze ..."
4,\nWięc bez obaw! W jego sieci\nbiedroneczka d...
...,...
353,"Z amforą wonnych olei w ręku\nChrystus, Pokora..."
354,"Czasem nęcą ją gwiazdy, czasem usta świeże,\nD..."
355,"I w brzóz idzie uśpione, białopienne gaje,\nKs..."
356,Mejmłodości i jasne powracasz miraże;\nMłodość...


In [ ]:
import re

annot = [
    'JANLEMAఞSKINokturn',
    'edytorski]',
    '[przypis redakcyjny]',
    'ANDRZEJNIEMOJEWSKIBranka',
    'MARYLAWOLSKAŚwięto Słońca',
    'ALEKSANDERREDROBajki',
    'LUDOMIఝKA',
    'ERYCZEK',
    'ANTONILANGESonety wedyckie',
    'Wszystkie materiały dodatkowe (przypisy, motywy literackie) są udostępnione na Licencji WolnejSztuki ..'
    'DUCHBOడY',
    'STANGWIAఠDZISTY',
    '(STANROśLINNY)',
    'KAZIMIERZRZERWA-TETMAJERAchilles',
    'RANCISZEKNOWICKIApoteoza',
    'TADEUSZNALEIఞSKIChrzest',
    'Wszystkie zasoby Wolnych Lektur możesz swobodnie wykorzystywać, publikować i rozpowszechniać pod wa-',
    'unkiem zachowania warunków licencji i zgodnie z Zasadami wykorzystania Wolnych Lektur.',
    'O-STATECZNOśఛ',
    'JANLEMAఞSKILis i gęś',
    'WACఝAWWOLSKIUczta upiorów',
    'MARYLAWOLSKASymfonia jesienna',
    'UCJANRYDELBajka o Kasi i królewiczu',
    'KAZIMIERZRZERWA-TETMAJERMuszla',
    'Ten utwór jest w domenie publicznej.',
    'ANDRZEJNIEMOJEWSKIKiedy kłamią',
    'EDWARDLESZCZYఞSKIKabaret szalony',
    'SZCZఝKA',
    'JULIANTUWIMRzepka'
]


def delete_unk(text):
    text = re.sub(r'[\uf700-\uf8ff]', '', text)
    if text == '':
        return text
    return text if text[0] != '\n' else text[1:]

def delete_annotations_and_problems(text):
    text = re.sub(r'^[¹²³⁴⁵⁶⁷⁸⁹⁰].*\n?', '', text, flags=re.MULTILINE)
    text = re.sub(r'[¹²³⁴⁵⁶⁷⁸⁹⁰]', '', text)
    text = re.sub(r'\b[MCDXLIV]+\b', '', text)
    text = re.sub(r'przypis edytorski', '', text)
    for a in annot:
        text = re.sub(re.escape(a), '', text)
    return text

def delete_flex(text):
    return delete_annotations_and_problems(delete_unk(text))

In [ ]:
df.text = df.text.map(delete_flex)

In [ ]:
df = df[df.text != '']
df

,text
0,"Pozwólcie, że wam przedstawię\nżółwia,\nco ma ..."
1,Kogut\n— Kuku! Ryku! Kukuryku‼! —\njuż od świt...
2,Już o świcie gna na łąkę —\nśmiało wita się z ...
3,"Dba o zdrowie należycie,\nco dzień drepcze na ..."
4,Więc bez obaw! W jego sieci\nbiedroneczka dziś...
...,...
353,"Z amforą wonnych olei w ręku\nChrystus, Pokora..."
354,"Czasem nęcą ją gwiazdy, czasem usta świeże,\nD..."
355,"w brzóz idzie uśpione, białopienne gaje,\nKsi..."
356,Mejmłodości i jasne powracasz miraże;\nMłodość...


In [ ]:
def split_poems_by_title(text):
    text = str(text)
    parts = re.split(r'\n(([A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]+)|([A-ZĄĆĘŁŃÓŚŹŻ]{2,}))\n', text)
    poems = []

    if len(parts) < 2:
        return [text.strip()]

    for i in range(1, len(parts), 2):
        title = parts[i].strip() if parts[i] else ""
        body = parts[i + 1].strip() if i + 1 < len(parts) and parts[i + 1] else ""
        poems.append(body)

    return poems

all_poems = []
for index, row in df.iterrows():
    poems = split_poems_by_title(row['text'])
    all_poems += poems

poems_df = pd.DataFrame(data={'text': all_poems})
poems_df

,text
0,Kotek
1,"Taki jest obyczajkoci,\nże swawoli wciąż i pso..."
2,Pies
3,"Wymyślili sobie ludzie,\nże pies musi mieszkać..."
4,Pies
...,...
863,"W jego zwierciedle ujrzeć cichą, jasną,\nRozko..."
864,Piekło
865,"Siekąc, krwawiąc nim trupie kochanków oblicze,..."
866,Pożądanie


In [ ]:
poems_df = poems_df[poems_df.text != '']

In [ ]:
poems_df['length'] = poems_df.text.map(lambda x: len(x.split('\n')))
poems_df.length.describe()

/var/folders/kq/k4c0ss5j4ync13791_11l7qr0000gn/T/ipykernel_32707/3166113416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poems_df['length'] = poems_df.text.map(lambda x: len(x.split('\n')))


count    831.000000
mean      12.275572
std       16.866303
min        1.000000
25%        1.000000
50%        5.000000
75%       17.000000
max      117.000000
Name: length, dtype: float64

In [ ]:
poems_df = poems_df[poems_df.length != 1]
poems_df

,text,length
1,"Taki jest obyczajkoci,\nże swawoli wciąż i pso...",19
3,"Wymyślili sobie ludzie,\nże pies musi mieszkać...",4
5,"w salonie, na dywanie,\nzrobił sobie w mig pos...",12
7,"— Do roboty, droga kuro,\nszybciej, prędzej, t...",16
9,"Prosiak sto posiada zalet,\na więc nie rozumie...",12
...,...,...
859,"w brzóz idzie uśpione, białopienne gaje,\nKsię...",17
861,"Niech zapomnę, że życie za mną i przede mną,\n...",12
863,"W jego zwierciedle ujrzeć cichą, jasną,\nRozko...",15
865,"Siekąc, krwawiąc nim trupie kochanków oblicze,...",5


### Literat

In [ ]:
links = [f'https://literat.ug.edu.pl/tetmajer/0{i if i > 9 else '0' + str(i)}.htm' for i in range(1, 91)]
links

['https://literat.ug.edu.pl/tetmajer/001.htm',
 'https://literat.ug.edu.pl/tetmajer/002.htm',
 'https://literat.ug.edu.pl/tetmajer/003.htm',
 'https://literat.ug.edu.pl/tetmajer/004.htm',
 'https://literat.ug.edu.pl/tetmajer/005.htm',
 'https://literat.ug.edu.pl/tetmajer/006.htm',
 'https://literat.ug.edu.pl/tetmajer/007.htm',
 'https://literat.ug.edu.pl/tetmajer/008.htm',
 'https://literat.ug.edu.pl/tetmajer/009.htm',
 'https://literat.ug.edu.pl/tetmajer/010.htm',
 'https://literat.ug.edu.pl/tetmajer/011.htm',
 'https://literat.ug.edu.pl/tetmajer/012.htm',
 'https://literat.ug.edu.pl/tetmajer/013.htm',
 'https://literat.ug.edu.pl/tetmajer/014.htm',
 'https://literat.ug.edu.pl/tetmajer/015.htm',
 'https://literat.ug.edu.pl/tetmajer/016.htm',
 'https://literat.ug.edu.pl/tetmajer/017.htm',
 'https://literat.ug.edu.pl/tetmajer/018.htm',
 'https://literat.ug.edu.pl/tetmajer/019.htm',
 'https://literat.ug.edu.pl/tetmajer/020.htm',
 'https://literat.ug.edu.pl/tetmajer/021.htm',
 'https://lit

In [ ]:
from bs4 import BeautifulSoup

def extract_text_from_literat_page(url):
    response = requests.get(url)
    response.encoding = 'iso-8859-2'
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    lines = [p.get_text(strip=True) for p in soup.find_all('p') if p.get_text(strip=True)]
    lines = list(dict.fromkeys(lines))

    return "\n".join(lines[:-1])

In [ ]:
poems = [extract_text_from_literat_page(poem_link) for poem_link in tqdm(links)]
df1 = pd.DataFrame(
    data = {'text': poems}
)
df1['length'] = df1.text.map(lambda x: len(x.split('\n')))
df1.length.describe()

100%|██████████| 90/90 [00:14<00:00,  6.12it/s]


count     90.000000
mean      34.488889
std       42.796420
min        8.000000
25%       14.000000
50%       20.000000
75%       39.500000
max      304.000000
Name: length, dtype: float64

### Poezja org

In [ ]:
pages_number = 100
links = ['https://poezja.org/wz/najpopularniejsze'] + [f'https://poezja.org/wz/najpopularniejsze/{i}/' for i in range(2, pages_number + 1)]

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_links_from_one_page_of_poezja_org(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    popular_section = soup.find('div', {'id': 'popular-texts'})
    links = popular_section.find_all('a', href=True)

    res = []
    for link in links:
        href = link['href']
        if href.strip() == '#':
            break
        elif str(href).strip()[-1] != '/':
            res.append(href)
    return res

poems_links = []
for base_link in tqdm(links):
    poems_links.extend(get_links_from_one_page_of_poezja_org(base_link))

100%|██████████| 100/100 [00:22<00:00,  4.54it/s]


In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_poem_with_br(url):
    try:
        response = requests.get(url)
    except Exception:
        return ''
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    poem_div = soup.find('div', itemprop='text')
    if not poem_div:
        return ''

    p = poem_div.find('p')
    if not p:
        return ''

    lines = []
    for element in p.children:
        if isinstance(element, str):
            lines.append(element.strip())
        elif element.name == 'br':
            continue
        else:
            lines.append(element.get_text(strip=True))
    lines = [line for line in lines if line]

    return "\n".join(lines)



In [ ]:
poems_text = [extract_poem_with_br(link) for link in tqdm(poems_links)]

100%|██████████| 4003/4003 [07:00<00:00,  9.53it/s]


In [ ]:
df2 = pd.DataFrame(
    data={
        'text': poems_text
    }
)

df2['length'] = df2.text.map(lambda x: len(x.split('\n')))
df2.length.describe()

count    4003.000000
mean       41.934299
std       112.057316
min         1.000000
25%        12.000000
50%        20.000000
75%        35.500000
max      2132.000000
Name: length, dtype: float64

In [ ]:
df2 = df2[df2.length > 4]
df2.length.describe()

count    3779.000000
mean       44.208256
std       114.929830
min         5.000000
25%        14.000000
50%        21.000000
75%        36.000000
max      2132.000000
Name: length, dtype: float64

In [ ]:
print(df2.text[400])

Gdy z wiosną życia duch artysta
Poi się jej tchem, jak motyle,
Wolno mu mówić tylko tyle:
«Ziemia jest krągła — jest kulista!»
Lecz, gdy późniejszych chłodów dreszcze
Drzewa wzruszą i kwiatki zlecą,
Wtedy dodawać trzeba jeszcze:
«U biegunów spłaszczona nieco...»
Ponad wszystkie wasze uroki,
Ty, poezjo, i ty, wymowo,
Jeden — wiecznie będzie wysoki:
Odpowiednie dać rzeczy słowo!


### Pisarzowiczka

In [ ]:
def extract_urls_from_link(url):
  res = set()
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " +
                    "AppleWebKit/537.36 (KHTML, like Gecko) " +
                    "Chrome/122.0.0.0 Safari/537.36"
  }

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
      soup = BeautifulSoup(response.text, "html.parser")
      all_links = [urljoin(url, a['href']) for a in soup.find_all('a', href=True)]
      matching_links = [link for link in all_links if link.startswith(url) and link != url]
      for link in matching_links:
          res.add(link)
  return res


def get_poems_urls_from_pisarzowicka():
  res = set()

  urls = [
      'https://pisarzowiczka.pl/bajki/',
      'https://pisarzowiczka.pl/poezja/'
  ]

  for url in urls:
    topic_links = extract_urls_from_link(url)
    for topic_link in topic_links:
      poems_links = extract_urls_from_link(topic_link)
      res.update(poems_links)
  return res

links = get_poems_urls_from_pisarzowicka()

In [ ]:
len(links)

157

In [ ]:
def extarct_text(url):
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " +
                    "AppleWebKit/537.36 (KHTML, like Gecko) " +
                    "Chrome/122.0.0.0 Safari/537.36"
  }

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')

      # Find the main content container. For this site, it's within a <div> with class "elementor-widget-container"
      text_divs = soup.find_all('div', class_='elementor-widget-container')

      full_text = ""
      for div in text_divs:
          text = div.get_text(separator="\n", strip=True)
          full_text += text + "\n"
          return full_text.strip()

  return ""

In [ ]:
texts = [extarct_text(url) for url in tqdm(links)]

100%|██████████| 157/157 [02:24<00:00,  1.09it/s]


In [ ]:
df3 = pd.DataFrame(
    data= {
        'text': texts
    }
)
df3
df3['length'] = df3.text.map(lambda x: len(x.split('\n')))
df3.length.describe()

count    157.000000
mean      21.828025
std       21.482903
min        1.000000
25%       12.000000
50%       14.000000
75%       18.000000
max      155.000000
Name: length, dtype: float64

In [ ]:
df = pd.concat([poems_df, df1, df2, df3], axis=0)
df

,text,length
1,"Taki jest obyczajkoci,\nże swawoli wciąż i pso...",19
3,"Wymyślili sobie ludzie,\nże pies musi mieszkać...",4
5,"w salonie, na dywanie,\nzrobił sobie w mig pos...",12
7,"— Do roboty, droga kuro,\nszybciej, prędzej, t...",16
9,"Prosiak sto posiada zalet,\na więc nie rozumie...",12
...,...,...
152,Pierwszy\ndzień trzeźwości.\nCzuję sporo mdłoś...,22
153,Człowiek nieraz jest jak morze:\nczasem o tej ...,12
154,"Jedni w sercu mają miłość,\ndrugim tylko bije,...",12
155,"Masz nierówno pod sufitem?\nWitaj w klubie, mó...",12


In [ ]:
df.length.describe()

count    4524.000000
mean       40.553050
std       105.797643
min         1.000000
25%        12.000000
50%        20.000000
75%        35.000000
max      2132.000000
Name: length, dtype: float64

In [ ]:
def split_text_by_lines(text, max_words=256, max_lines_count = 32):
    lines = text.split('\n')
    parts = []
    current_part = []
    current_word_count = 0
    line_counter = 0

    for line in lines:
        line_counter += 1
        word_count = len(line.split())
        if current_word_count + word_count <= max_words and line_counter <= max_lines_count:
            current_part.append(line)
            current_word_count += word_count
        else:
            parts.append(['\n'.join(current_part), current_word_count])
            current_part = [line]
            current_word_count = word_count
            line_counter = 1

    if current_part:
        parts.append(['\n'.join(current_part), current_word_count])

    return parts

rows = []
for index, row in df.iterrows():
    chunks = split_text_by_lines(row['text'])
    for part, count in chunks:
        rows.append({'text': part, 'length': count})

df_split = pd.DataFrame(rows)
df_split

,text,length
0,"Taki jest obyczajkoci,\nże swawoli wciąż i pso...",84
1,"Wymyślili sobie ludzie,\nże pies musi mieszkać...",19
2,"w salonie, na dywanie,\nzrobił sobie w mig pos...",62
3,"— Do roboty, droga kuro,\nszybciej, prędzej, t...",73
4,"Prosiak sto posiada zalet,\na więc nie rozumie...",54
...,...,...
8600,Pierwszy\ndzień trzeźwości.\nCzuję sporo mdłoś...,48
8601,Człowiek nieraz jest jak morze:\nczasem o tej ...,57
8602,"Jedni w sercu mają miłość,\ndrugim tylko bije,...",45
8603,"Masz nierówno pod sufitem?\nWitaj w klubie, mó...",65


In [ ]:
import regex

def filter_text(text):
    text = str(text)
    pattern = regex.compile(r'^[\p{L}\p{P}\p{Zs}]+$')
    for line in text.splitlines():
        if not pattern.match(line):
            print(line)


    filtered_lines = [line for line in text.splitlines() if pattern.match(line)]

    return '\n'.join(filtered_lines)

df_clean = df_split.copy()
df_clean.text = df_clean.text.map(lambda x: filter_text(x))
sum(df_clean.text.map(lambda x: len(x.split("\n"))))

O! Na uszach ma ędzelki!
jada też ancuskie sery,
ytki, bitki i kotlety.
Nie uwam wprawdzie jak sójka,










Ale je same tylko zajmowały aszki;
cacko wyymarczali najdroższe zapasy życia… a mało bardzo było takich, coby donieśli do
nad przyszłym jego losem, asując się, co z niego będzie.
Cały tłum stojący dokoła zaasowane bardzo miał twarze; niektórzy ręce łamali i roz-
Gwoździk, poczciwe człowieczysko, koronę rzuciwszy do kua i pozbywszy ceremo-
W siódmym dębowe stoły i sza,
W ósmym słoń, niedźwiedź i dwie żyra,
W dziesiątym — kuy, paki i skrzynie.
Lecz aszka, igraszka, zabawka blaszana.
W siódmym dębowe stoły i sza,
W ósmym słoń, niedźwiedź i dwie żyra,
W dziesiątym — kuy, paki i skrzynie,
Lecz aszka, igraszka, zabawka blaszana.

Bo sunęły się ptaszki dla odbycia narad:
„Gdzieś ty latał? Gdzieś ty uwał? Przecież ja tu płaczę!”
Maca szlaok, palto maca.
Maca szlaok, palto maca.
Bo sunęły się ptaszki dla odbycia narad:
Przyunęła ptasia milicja







z tysiąca i

182088

In [ ]:
def clean_line(line):
    return re.sub(r'\s*\[\s*\]\s*', '', line)

def clean_word(text):
    text = re.sub(r'[\[\(\{\«\"\']+', '', text)
    text = re.sub(r'[\]\)\}\»\"\',:;.!?…\-—⁈]+', '', text)
    words = re.findall(r'\w{2,}', text)
    return words[-1] if words else ''

def add_first_word_to_line(text):
    res = []
    lines = text.split('\n')
    for line in lines:
        line = clean_line(line).strip()
        if not line:
            continue
        keyword = clean_word(line)
        if not keyword:
            continue
        extra_token = f'[{keyword}]'
        res.append(f'{extra_token} {line}')
    return '\n'.join(res)

In [ ]:
df_first_word_explicit = df_clean.copy()
df_first_word_explicit['text'] = df_first_word_explicit['text'].map(add_first_word_to_line)

In [ ]:
df_first_word_explicit.length = df_first_word_explicit.text.map(lambda x: len(x.split('\n')))
df_first_word_explicit = df_first_word_explicit[df_first_word_explicit.length > 3]
df_first_word_explicit.reset_index(inplace=True)
df_first_word_explicit = df_first_word_explicit.loc[:, ['text', 'length']]
df_first_word_explicit

,text,length
0,"[obyczajkoci] Taki jest obyczajkoci,\n[psoci] ...",19
1,"[ludzie] Wymyślili sobie ludzie,\n[budzie] że ...",4
2,"[dywanie] w salonie, na dywanie,\n[posłanie] z...",12
3,"[kuro] — Do roboty, droga kuro,\n[biuro] szybc...",16
4,"[zalet] Prosiak sto posiada zalet,\n[wcale] a ...",12
...,...,...
8146,[Pierwszy] Pierwszy\n[trzeźwości] dzień trzeźw...,22
8147,[morze] Człowiek nieraz jest jak morze:\n[porz...,12
8148,"[miłość] Jedni w sercu mają miłość,\n[bije] dr...",12
8149,[sufitem] Masz nierówno pod sufitem?\n[kolego]...,12


In [ ]:
df_first_word_explicit.to_csv('polskie_wiersze.csv', index=False)

## Data preporation

In [ ]:
data = pd.read_csv('polskie_wiersze.csv')
data

,text,length
0,"[obyczajkoci] Taki jest obyczajkoci,\n[psoci] ...",19
1,"[ludzie] Wymyślili sobie ludzie,\n[budzie] że ...",4
2,"[dywanie] w salonie, na dywanie,\n[posłanie] z...",12
3,"[kuro] — Do roboty, droga kuro,\n[biuro] szybc...",16
4,"[zalet] Prosiak sto posiada zalet,\n[wcale] a ...",12
...,...,...
33403,"[środku] W pewnej krypcie, w samym środku,\n[n...",32
33404,"[głupawa] Mała, krwista i głupawa.\n[sprawa] „...",32
33405,[nadziei] pełen lęku i nadziei.\n[kolei] Wszys...,14
33406,[długo] Trenowaliśmy więź długo –\n[zasługą] z...,14


In [ ]:
data.text = data.text.astype(str)
system_prompt = "Napisz wiersz. Każdy wers powinien kończyć się podanym słowem [w nawiasie kwadratowym].\n"

data.text = data.text.map(lambda x: system_prompt + x)
data

,text,length
0,Napisz wiersz. Każdy wers powinien kończyć się...,19
1,Napisz wiersz. Każdy wers powinien kończyć się...,4
2,Napisz wiersz. Każdy wers powinien kończyć się...,12
3,Napisz wiersz. Każdy wers powinien kończyć się...,16
4,Napisz wiersz. Każdy wers powinien kończyć się...,12
...,...,...
33403,Napisz wiersz. Każdy wers powinien kończyć się...,32
33404,Napisz wiersz. Każdy wers powinien kończyć się...,32
33405,Napisz wiersz. Każdy wers powinien kończyć się...,14
33406,Napisz wiersz. Każdy wers powinien kończyć się...,14


In [ ]:
dataset = Dataset.from_dict({"text": data.text})
dataset

Dataset({
    features: ['text'],
    num_rows: 33408
})

In [ ]:
# model_checkpoint = "sdadas/polish-gpt2-small"
model_checkpoint = "AndreiRabau/poem-gpt-s"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/559k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.81M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [ ]:
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/33408 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

In [ ]:
train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 30067
})

## Train process

# New Section

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    num_train_epochs=4,
    report_to = 'none'
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-13-1db6836bb964>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.772500,2.655733
2,2.362600,2.447483
3,1.998800,2.321740
4,1.677700,2.272517


TrainOutput(global_step=15036, training_loss=2.2609218238166875, metrics={'train_runtime': 10297.0297, 'train_samples_per_second': 11.68, 'train_steps_per_second': 1.46, 'total_flos': 1.5712535052288e+16, 'train_loss': 2.2609218238166875, 'epoch': 4.0})

In [ ]:
!pip install transformers huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `poems` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [ ]:
model.push_to_hub("poem-gpt-s")
tokenizer.push_to_hub("poem-gpt-s")